In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
seed_url = 'http://www.census.nationalarchives.ie/pages/1911/Down/Kilkeel/'
request = requests.get(seed_url)
soup = BeautifulSoup(request.content, "html.parser")

townland_url_extensions = []
seed_url_extension = seed_url.replace(
    'http://www.census.nationalarchives.ie', '')
for link in soup.find_all('a'):
    if link.get('href').startswith(seed_url_extension):
        if link.get('href') != seed_url_extension:
            townland_url_extensions.append(link.get('href'))
            
townland_url_extensions

In [ ]:
house_urls = []

for url_extension in townland_url_extensions:
    request = requests.get('http://www.census.nationalarchives.ie' 
                           + url_extension)
    soup = BeautifulSoup(request.content, "html.parser")
    
    for link in soup.find_all('a'):
        if link.get('href').startswith(url_extension):
            if link.get('href') != (url_extension):
                house_urls.append('http://www.census.nationalarchives.ie' +
                                  link.get('href'))
                
house_urls = list(set(house_urls))

house_urls

In [ ]:
residents = []

for url in house_urls:
    split_url = url.split('/')
    townland_or_street = split_url[-3]
    house_id = split_url[-2]
    
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "html.parser")
    
    heading = (str(soup.find_all('h1')[0]))
    trimmed_heading = heading.replace('<h1>Residents of a house ', '')
    split_heading = trimmed_heading.split(' ', 1)
    house = split_heading[0]
    house

    table = soup.find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    resident_number = 1
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        cols.insert(0, resident_number)
        cols.insert(0, house)
        cols.insert(0, townland_or_street)
        cols.insert(0, house_id)
        residents.append([ele for ele in cols])
        resident_number += 1

column_headers_1901 = ['house_id', 'townland_or_street', 'house',
                  'resident_number', 'surname', 'forename', 'age', 'sex', 
                  'relation_to_head', 'religion', 'birthplace', 'occupation', 
                  'literacy', 'irish_language', 'marital_status', 
                  'specified_illnesses']

column_headers_1911 = ['house_id', 'townland_or_street', 'house',
                  'resident_number', 'surname', 'forename', 'age', 'sex', 
                  'relation_to_head', 'religion', 'birthplace', 'occupation', 
                  'literacy', 'irish_language', 'marital_status', 
                  'specified_illnesses', 'years_married', 'children_born',
                  'children_living']

year = seed_url.split('/')[-4]
county = seed_url.split('/')[-3]
ded = seed_url.split('/')[-2]

if year == '1901':
    seed_DED_df = pd.DataFrame(residents, columns=column_headers_1901)

if year == '1911':
    seed_DED_df = pd.DataFrame(residents, columns=column_headers_1911)

seed_DED_df.insert(1, 'ded', ded)
seed_DED_df.insert(1, 'county', county)

seed_DED_df = seed_DED_df.sort_values(by=['house_id', 'resident_number'])
seed_DED_df = seed_DED_df.reset_index(drop='True')

seed_DED_df

In [ ]:
seed_DED_df.to_csv(year + '_' + county + '_' + ded + '_raw.csv')